In [1]:
import torch
from torchvision.datasets import ImageFolder
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
from datetime import datetime
from torchvision import transforms as tfs
from torch.nn import functional as F

In [2]:
data_tf = tfs.ToTensor()

In [3]:
train_set = ImageFolder('./Fruit-Images-Dataset/Training', transform = data_tf)

In [4]:
test_set = ImageFolder('./Fruit-Images-Dataset/Test', transform = data_tf)

In [5]:
train_data = DataLoader(train_set, batch_size = 32, shuffle = True)
test_data = DataLoader(test_set, batch_size = 256, shuffle = False)

In [6]:
#第一个模型
class conv_net1(nn.Module):
    def __init__(self):
        super(conv_net1, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.max_pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.max_pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.batch_norm3 = nn.BatchNorm2d(64)
        self.max_pool3 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6400, 3000)
        self.fc2 = nn.Linear(3000, 1000)
        self.fc3 = nn.Linear(1000, 101)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = self.max_pool3(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [7]:
net = conv_net1()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [8]:
# def get_acc(output, label):
#     total = output.shape[0]
#     _, pred_label = output.max(1)
#     num_correct = (pred_label == label).sum().data.item()
#     return num_correct / total


def train(net, train_data, test_data, num_epochs, optimizer, criterion):
    if torch.cuda.is_available():
        net = net.cuda()
    prev_time = datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0
        train_acc = 0
        net = net.train()
        for im, label in train_data:
            if torch.cuda.is_available():
                im = im.cuda() 
                label = label.cuda() 
                
            # forward
            output = net(im)
            loss = criterion(output, label)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()
            _,pred = output.max(1)
            correct_num = (pred == label).sum().data.item()
            acc = correct_num / im.shape[0]
            train_acc += acc

        cur_time = datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if test_data is not None:
            test_loss = 0
            test_acc = 0
            net = net.eval()
            for im, label in test_data:
                if torch.cuda.is_available():
                    im = im.cuda()
                    label = label.cuda()

                output = net(im)
                loss = criterion(output, label)
                test_loss += loss.data.item()
                _, pred = output.max(1)
                num_correct = (pred == label).sum().data.item()
                acc = num_correct / im.shape[0]
                test_acc += acc
            epoch_str = (
                "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), test_loss / len(test_data),
                  test_acc / len(test_data)))
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %
                         (epoch, train_loss / len(train_data),
                          train_acc / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str)

In [ ]:
train(net, train_data, test_data, 3, optimizer, criterion)

In [9]:
class conv_net2(nn.Module):
    def __init__(self):
        super(conv_net2,self).__init__()
        self.conv = nn.Conv2d(3, 16, 3, 5)
        self.batch_norm = nn.BatchNorm2d(16)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(1600, 101)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.max_pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [10]:
net = conv_net2()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

In [11]:
train(net, train_data, test_data, 3, optimizer, criterion)

Epoch 0. Train Loss: 1.009434, Train Acc: 0.815311, Valid Loss: 0.575515, Valid Acc: 0.845788, Time 00:00:38
Epoch 1. Train Loss: 0.105616, Train Acc: 0.990055, Valid Loss: 0.677816, Valid Acc: 0.799592, Time 00:00:50
Epoch 2. Train Loss: 0.047352, Train Acc: 0.996672, Valid Loss: 0.329280, Valid Acc: 0.899004, Time 00:00:50
